## Архитектура MLflow

### Компонтенты MLflow и их задачи

 - MLflow Project - создание среды для экспериментов, группировка экспериментов.
 - MLflow Tracking - фиксация параметров и метрик качества экспериментов.
 - MLflow Models - подготовка версии модели к дистрибуции.
 - MLflow Registry - централизованное хранилище моделей и выкладка в эксплуатацию.

#### MLflow Projects

- Настройка окружения:
	- языки программирования
	- менеджер пакетов (например, conda)
	- зависимости (библиотеки xgboost, scikit-learn, ...)

- Описание окружения (Infrastructure as code):
	- различные ОС
	- локальное окружение
	- облачные сервисы

#### MLflow Tracking

Фиксирует все, что связано с запуском модели:
 - Наборы данных (для обучения и тестирования)
 - Наборы параметров (например, количество деревьев, слоев, L1/L2)
 - Значения метрик качества
 - Скорость работы и прочие технические метрики

#### MLflow Models

Cериализует артефакты моделей устраняя необходимость дополнительной разработки

#### MLflow Registry

 - Централизованное хранение версий моделей для удобного поиска.
 - Информация о том, какая модель используется на какой среде
 - История всех версий и их использования в средах



In [2]:
%%writefile MLproject
name: tutorial

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"

Overwriting MLproject


In [3]:
%%writefile conda.yaml
name: tutorial
channels:
  - defaults
dependencies:
  - numpy>=1.14.3
  - pandas>=1.0.0
  - scikit-learn=0.19.1
  - pip
  - pip:
    - mlflow

Writing conda.yaml


дата сет можно найти здесь https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009

In [4]:
import os
import sys
import warnings
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

MLFLOW_SERVER_URL = 'http://127.0.0.1:5000/'

warnings.filterwarnings("ignore")
np.random.seed(40)
data = pd.read_csv("wine-quality.csv")

train, test = train_test_split(data)

train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

### Создание и запуск эксперимента

Код самого эксперимента не зависит от MLflow, можно использовать уже готовый код.

Чтобы зафиксировать параметры запуска и метрики модели, нужно запустить обучение в рамках эксперимента и проекта.

`tracking_url` - адрес поднятого `mlflow` сервера, который будет использоваться для хранения экспериментов. Также по этому адресу доступен веб-интерфейс для просмотра результатов запусков.

In [5]:
# подключаемся к серверу
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'experiment2'
mlflow.set_experiment(experiment_name)

# запуск в эксперименте
with mlflow.start_run():
    alpha = 0.5
    l1_ratio = 0.5

    # модель
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # метрики
    predicted_qualities = lr.predict(test_x)
    rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))
    mae = mean_absolute_error(test_y, predicted_qualities)
    r2 = r2_score(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # сохраняем значения эксперимента в системе
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

INFO: 'experiment2' does not exist. Creating a new experiment
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284976
  MAE: 0.627876141016
  R2: 0.126787219728


Запуск успешного эксперимента можно подготовить к вводу в эксплуатацию.

Для этого используется  `MLflow Model Registry`.

### Просмотр проведенных экспериментов и выбор кандидата на ввод в эксплуатацию

Чтобы получить информацию о запусках экспериментов, нужно создать клиент `mlflow.tracking.MlflowClient`, затем выбрать нужный эксперимент и выбрать искомый запуск эксперимента.

В коде ниже берется последний запуск эксперимента из списка всех запусков.

In [6]:
client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)
run_info = client.list_run_infos(experiment.experiment_id)[-1]

print(experiment)
print(run_info)

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='experiment2', tags={}>
<RunInfo: artifact_uri='./mlruns/1/fc32649252bc47388aaf8c46a9b69b1b/artifacts', end_time=1605466453616, experiment_id='1', lifecycle_stage='active', run_id='fc32649252bc47388aaf8c46a9b69b1b', run_uuid='fc32649252bc47388aaf8c46a9b69b1b', start_time=1605466453301, status='FINISHED', user_id='jovyan'>


### Регистрация модели в MLflow Model Registry

Регистрация модели также доступна в веб-интерфейсе. Для этого нужно выбрать модель на странице экспериментов и нажать `Register Model`.

Ниже приведен код, совершающий аналогичное действие.

In [7]:
reg_model_name = "sk-learn-new-model"

# региструем модель
client.create_registered_model(reg_model_name)
# создаем первую версию
result = client.create_model_version(
    name=reg_model_name,
    source=f"{run_info.artifact_uri}/model",
    run_id=run_info.run_id
)

print(result)

2020/11/15 18:54:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-new-model, version 1


<ModelVersion: creation_timestamp=1605466453678, current_stage='None', description='', last_updated_timestamp=1605466453678, name='sk-learn-new-model', run_id='fc32649252bc47388aaf8c46a9b69b1b', run_link='', source='./mlruns/1/fc32649252bc47388aaf8c46a9b69b1b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


Посмотреть список зарегистрированных моделей можно следующим образом:

In [8]:
for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)

{   'creation_timestamp': 1605466453668,
    'description': '',
    'last_updated_timestamp': 1605466453678,
    'latest_versions': [   <ModelVersion: creation_timestamp=1605466453678, current_stage='None', description='', last_updated_timestamp=1605466453678, name='sk-learn-new-model', run_id='fc32649252bc47388aaf8c46a9b69b1b', run_link='', source='./mlruns/1/fc32649252bc47388aaf8c46a9b69b1b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': 'sk-learn-new-model',
    'tags': {}}


Найти нужную модель и версию можно с помощью специальной функции:

In [9]:
for mv in client.search_model_versions(f"name='{reg_model_name}'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1605466453678,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1605466453678,
    'name': 'sk-learn-new-model',
    'run_id': 'fc32649252bc47388aaf8c46a9b69b1b',
    'run_link': '',
    'source': './mlruns/1/fc32649252bc47388aaf8c46a9b69b1b/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


### Выкладка модели в тестовую и эксплуатационную среды

Терминология, использующаяся в MLflow Model Registry:

 - Staging - тестовая среда
 - Production - эксплуатационная среда
 
Следующий код переводит ранее зарегистрированную модель версии `1` в тестовую среду (`Staging`).

In [10]:
client.transition_model_version_stage(
    name=reg_model_name,
    version=1,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1605466453678, current_stage='Staging', description='', last_updated_timestamp=1605466453716, name='sk-learn-new-model', run_id='fc32649252bc47388aaf8c46a9b69b1b', run_link='', source='./mlruns/1/fc32649252bc47388aaf8c46a9b69b1b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

Проверить результат можно следующим кодом:

In [11]:
for mv in client.search_model_versions(f"name='{reg_model_name}'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1605466453678,
    'current_stage': 'Staging',
    'description': '',
    'last_updated_timestamp': 1605466453716,
    'name': 'sk-learn-new-model',
    'run_id': 'fc32649252bc47388aaf8c46a9b69b1b',
    'run_link': '',
    'source': './mlruns/1/fc32649252bc47388aaf8c46a9b69b1b/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


Чтобы использовать модель, нужно запустить веб-сервер, в качестве параметров передать ему название модели и среды, а также порт
Для взаимодействия с веб-сервером следует отправлять запросы к точке входа `/invocations` этого веб-сервера.

In [12]:
os.system('MLFLOW_TRACKING_URI=http://127.0.0.1:5000/ mlflow models serve -m "models:/sk-learn-new-model/Staging" -p 5005 --no-conda &')

0

In [14]:
import requests

url = f'http://127.0.0.1:5005/invocations'

http_data = test_x[:10].to_json(orient='split')
response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')

Predictions: [5.88527611335971, 5.740523335804658, 5.811935039818762, 5.957850132893952, 6.092739574397946, 5.381768818172121, 6.050858460676247, 5.992469597297116, 5.850426199303677, 5.627508796953737]
